In [1]:
import pandas as pd
import numpy as np
import sklearn.metrics

importer les donnees du titanic et afficher les 5 premieres lignes

In [2]:
train = pd.read_csv('/home/aaron/3A-Prog/TP - PDS - New/TP - PDS/13-Logistic-Regression/titanic_train.csv')
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


faire l'inventaire des colonnes du dataframe en precisant les colonnes quantitatives et les qualitatives

5 colonnes qualitatives
6 quantitatives

identifier les valeurs manquantes et imputer les par le mode pour les variables qualitatives et la mediane pour les varialbes quantitatives

In [3]:
train.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [4]:
train['Age'] = train['Age'].fillna(value=train['Age'].median())

In [5]:
train.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            891
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [6]:
train['Cabin'] = train['Cabin'].fillna(value=train['Cabin'].mode()[0])

In [7]:
train['Cabin'].mode()

0    B96 B98
dtype: object

In [8]:
train.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            891
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          891
Embarked       889
dtype: int64

considerer les variables ['SibSp', 'Parch', 'Fare'] et mettre en place un modèle de regression logistique

In [9]:
#Convertir les int en object / Façon prof avec un tableau
def convert_int_to_object(df, cols_name):

    for cols in cols_name:
        df[cols] = df[cols].astype("object")
    return df
        
cols_name=['Survived','Pclass']
train = convert_int_to_object(train, cols_name)

In [10]:
dead = train[train['Survived']==0]
survived = train[train['Survived']==1]

creer la target qui est represente par la colonne survived
combien de personnes ont survecu?
combien de femmes ont embarqué et combien ont survecu?
survived = train[train.Survived == 1]
dead = train[train.Survived == 0]

In [11]:

def getTitle(str):
    return str.split(',')[1].split('.')[0].strip()

train['Title'] = train['Name'].apply(lambda x : getTitle(x))
train['Surname'] = train['Name'].apply(lambda x : '(' in x)

In [12]:
def input_missing_values(df):
    for col in df.columns:
        if (df[col].dtype is float) or (df[col].dtype is int):
            df[col] = df[col].fillna(df[col].median())
        if (df[col].dtype == object):
            df[col] = df[col].fillna(df[col].mode()[0])

    return df
a = input_missing_values(train)


### quick modelisation: logistics Regression 



In [13]:
def parse_model(X, use_columns):
    if "Survived" not in X.columns:
        raise ValueError("target column survived should belong to df")
    target = X["Survived"]
    X = X[use_columns]
    return X, target

model_cols1 =['SibSp', 'Fare','Parch', 'Pclass']
X, y = parse_model(X=train.copy(), use_columns=model_cols1)

In [14]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

Reg_Log = LogisticRegression(random_state=42)
Reg_Log.fit(X_train, y_train)
ypred = Reg_Log.predict(X_test)
print('Score : ',(ypred==y_test).sum()/ypred.size)

Score :  0.7052238805970149


/home/aaron/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [16]:
X_dummies = pd.get_dummies(X['Pclass'], prefix="Pclass", drop_first=False,dummy_na=False)
x = X.join(X_dummies).drop("Pclass", axis=1)
x

,SibSp,Fare,Parch,Pclass_1,Pclass_2,Pclass_3
0,1,7.2500,0,0,0,1
1,1,71.2833,0,1,0,0
2,0,7.9250,0,0,0,1
3,1,53.1000,0,1,0,0
4,0,8.0500,0,0,0,1
5,0,8.4583,0,0,0,1
6,0,51.8625,0,1,0,0
7,3,21.0750,1,0,0,1
8,0,11.1333,2,0,0,1
9,1,30.0708,0,0,1,0


In [17]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=42)

Reg_Log = LogisticRegression(random_state=42)
Reg_Log.fit(X_train, y_train)
ypred = Reg_Log.predict(X_test)
print('Score : ',(ypred==y_test).sum()/ypred.size)

Score :  0.7425373134328358


/home/aaron/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [18]:
x = x.join(train['Age'])
x

,SibSp,Fare,Parch,Pclass_1,Pclass_2,Pclass_3,Age
0,1,7.2500,0,0,0,1,22.0
1,1,71.2833,0,1,0,0,38.0
2,0,7.9250,0,0,0,1,26.0
3,1,53.1000,0,1,0,0,35.0
4,0,8.0500,0,0,0,1,35.0
5,0,8.4583,0,0,0,1,28.0
6,0,51.8625,0,1,0,0,54.0
7,3,21.0750,1,0,0,1,2.0
8,0,11.1333,2,0,0,1,27.0
9,1,30.0708,0,0,1,0,14.0


##### Note: For integer/None inputs, if the estimator is a classifier and y is either binary or multiclass, StratifiedKFold is used. In all other cases, KFold is used.

In [19]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression

In [20]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=42)

Reg_Log = LogisticRegression(random_state=42)
Reg_Log.fit(X_train, y_train)
ypred = Reg_Log.predict(X_test)
print('Score : ',(ypred==y_test).sum()/ypred.size)

Score :  0.7164179104477612


/home/aaron/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [21]:
x.drop("Age", axis=1)

,SibSp,Fare,Parch,Pclass_1,Pclass_2,Pclass_3
0,1,7.2500,0,0,0,1
1,1,71.2833,0,1,0,0
2,0,7.9250,0,0,0,1
3,1,53.1000,0,1,0,0
4,0,8.0500,0,0,0,1
5,0,8.4583,0,0,0,1
6,0,51.8625,0,1,0,0
7,3,21.0750,1,0,0,1
8,0,11.1333,2,0,0,1
9,1,30.0708,0,0,1,0


In [22]:
X = x.copy()
X = X.drop("Age", axis=1)

Essaie avec la classe Agebin, séparant les personnes en classe d'âge

In [23]:
train["Agebin"] = pd.cut(train["Age"],bins=[0,8,60,85], labels=[1,2,3]) #classes d'age
X = X.join(train["Agebin"])

In [24]:
W = X.copy()
W = W.join(train["Sex"])

In [25]:
W_dummies = pd.get_dummies(W['Sex'], prefix="Sex", drop_first=False,dummy_na=False)
W = W.join(W_dummies).drop("Sex", axis=1)
W

,SibSp,Fare,Parch,Pclass_1,Pclass_2,Pclass_3,Agebin,Sex_female,Sex_male
0,1,7.2500,0,0,0,1,2,0,1
1,1,71.2833,0,1,0,0,2,1,0
2,0,7.9250,0,0,0,1,2,1,0
3,1,53.1000,0,1,0,0,2,1,0
4,0,8.0500,0,0,0,1,2,0,1
5,0,8.4583,0,0,0,1,2,0,1
6,0,51.8625,0,1,0,0,2,0,1
7,3,21.0750,1,0,0,1,1,0,1
8,0,11.1333,2,0,0,1,2,1,0
9,1,30.0708,0,0,1,0,2,1,0


In [26]:
X_train, X_test, y_train, y_test = train_test_split(W, y, test_size=0.3,random_state=42)

Reg_Log = LogisticRegression(random_state=42)
Reg_Log.fit(X_train, y_train)
ypred = Reg_Log.predict(X_test)
print('Score : ',(ypred==y_test).sum()/ypred.size)

Score :  0.8022388059701493


/home/aaron/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Avec Agebin et Sex, on atteint un score de 81%, essayons avec un colonne "is_child", en mettant un pallier à 8 ans.

In [27]:
train.loc[(train.Sex == "male", "Sex")]=1
train.loc[(train.Sex == "female", "Sex")]=0 

W = X.copy()
W

,SibSp,Fare,Parch,Pclass_1,Pclass_2,Pclass_3,Agebin
0,1,7.2500,0,0,0,1,2
1,1,71.2833,0,1,0,0,2
2,0,7.9250,0,0,0,1,2
3,1,53.1000,0,1,0,0,2
4,0,8.0500,0,0,0,1,2
5,0,8.4583,0,0,0,1,2
6,0,51.8625,0,1,0,0,2
7,3,21.0750,1,0,0,1,1
8,0,11.1333,2,0,0,1,2
9,1,30.0708,0,0,1,0,2


In [28]:
def is_child(float):
    if float <= 8:
        return 1
    else:
        return 0

train['Is_Child'] = train['Age'].apply(lambda x : is_child(x))

In [30]:
W = W.join(train["Is_Child"])

In [31]:
X_train, X_test, y_train, y_test = train_test_split(W, y, test_size=0.3,random_state=42)

Reg_Log = LogisticRegression(random_state=42)
Reg_Log.fit(X_train, y_train)
ypred = Reg_Log.predict(X_test)
print('Score : ',(ypred==y_test).sum()/ypred.size)

Score :  0.7126865671641791


/home/aaron/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Le score est globalement le même avec les deux techniques: 81%.

In [32]:
X = W.copy()

In [ ]:
train.groupby(["Title"]).mean()

In [42]:
#Merge des titres qui sont juste des traductions
def merge_title(str):
    if str == "Miss" or str == "Mlle" or str == "Ms":
        return "Miss"
    elif str == "Mrs" or str == "Mme":
        return "Mrs"
    elif str == "Master":
        return "Master"
    else:
        return "Rare"

#Calcul du taux de survit par titre
train['Title'] = train['Title'].apply(lambda x : merge_title(x))
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Miss,0.702703
1,Mrs,0.793651
2,Rare,0.193103


In [43]:
#Mapping des titres
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
train['Title'] = train['Title'].map(title_mapping)
train['Title'] = train['Title'].fillna(0)
X = X.join(train["Title"])
X

,SibSp,Fare,Parch,Pclass_1,Pclass_2,Pclass_3,Agebin,Is_Child,Title
0,1,7.2500,0,0,0,1,2,0,5
1,1,71.2833,0,1,0,0,2,0,3
2,0,7.9250,0,0,0,1,2,0,2
3,1,53.1000,0,1,0,0,2,0,3
4,0,8.0500,0,0,0,1,2,0,5
5,0,8.4583,0,0,0,1,2,0,5
6,0,51.8625,0,1,0,0,2,0,5
7,3,21.0750,1,0,0,1,1,1,5
8,0,11.1333,2,0,0,1,2,0,3
9,1,30.0708,0,0,1,0,2,0,3


In [58]:
model_cols2 =['SibSp', 'Parch', 'Fare', 'Pclass','Sex','Is_Child','Title']
X2, y2 = parse_model(X=train.copy(), use_columns=model_cols2)

In [ ]:
#Ancien Test concernant les Titre, les dummies font beaucoup de features pour assez peu d'intérêt
X_dummies = pd.get_dummies(X['New_Title'], prefix="New_Title", drop_first=False,dummy_na=False)
X = X.join(X_dummies).drop("New_Title", axis=1)
X

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

Reg_Log = LogisticRegression(random_state=42)
Reg_Log.fit(X_train, y_train)
ypred = Reg_Log.predict(X_test)
print('Score : ',(ypred==y_test).sum()/ypred.size)

Score :  0.8059701492537313


/home/aaron/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
train['Surname'] = train['Name'].apply(lambda x : '(' in x)
mask = train['Surname']==True
train.loc[mask, ["Name", "Surname"]]

In [78]:
#RandomForest
from sklearn.ensemble import *
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y, test_size=0.3,random_state=42)
random_forest = RandomForestClassifier(n_estimators=150)
random_forest.fit(X_train2, y_train2)
Y_pred = random_forest.predict(X_test2)
random_forest.score(X_test2, y_test2)
acc_random_forest = round(random_forest.score(X_test2, y_test2) * 100, 2)
acc_random_forest

80.6

In [50]:
X2.columns

Index(['SibSp', 'Parch', 'Fare', 'Pclass', 'Sex', 'Is_Child', 'Title'], dtype='object')

In [76]:
from sklearn.model_selection import GridSearchCV

Estimator = RandomForestClassifier(random_state=42)
parameters = {
    'n_estimators': [100,150,200,250,300],
    'max_depth': np.arange(6,16,2),
    'min_samples_split': np.arange(10,30,5),
    'min_samples_leaf': np.arange(5,20,5)
}

model2 = GridSearchCV

# cross_val_score(estimator=Estimator, X=X_train, y=y_train, cv=5)
gd_sr = GridSearchCV(estimator=Estimator,
                     param_grid=parameters,
                     scoring='accuracy',
                     verbose=1,
                     cv=5,
                     n_jobs=-1)
gd_sr.fit(X_train2, y_train2)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 352 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 852 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:   49.2s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'ma

In [77]:
gd_sr.best_params_

{'max_depth': 6,
 'min_samples_leaf': 5,
 'min_samples_split': 15,
 'n_estimators': 150}

In [54]:
random_forest = RandomForestClassifier(max_depth=6,min_samples_leaf=5, min_samples_split=15,n_estimators=150)
random_forest.fit(X_train2, y_train2)
Y_pred = random_forest.predict(X_test2)
random_forest.score(X_test2, y_test2)
acc_random_forest = round(random_forest.score(X_test2, y_test2) * 100, 2)
acc_random_forest

80.97

### Etude des variables

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
%matplotlib inline
%pylab inline


def plot_hist(feature, bins=20):
    x1 = np.array(dead[feature].dropna())
    x2 = np.array(survived[feature].dropna())
    plt.hist([x1, x2], label=["Victime", "Survivant"], bins=bins, color=['r', 'b'])
    plt.legend(loc="upper left")
    plt.title('distribution relative de %s' %feature)
    plt.show()


In [ ]:
from sklearn import preprocessing

def dummify_features(df):
    """
    Transform categorical variables to dummy variables.

    Parameters
    ----------
    df: dataframe containing only categorical features

    Returns
    -------
    X: new dataframe with dummified features
       Each column name becomes the previous one + the modality of the feature

    enc: the OneHotEncoder that produced X (it's used later in the processing chain)
    """
    colnames = df.columns
    le_dict = {}
    for col in colnames:
        le_dict[col] = preprocessing.LabelEncoder()
        le_dict[col].fit(df[col])
        df.loc[:, col] = le_dict[col].transform(df[col])

    enc = preprocessing.OneHotEncoder()
    enc.fit(df)
    X = enc.transform(df)

    dummy_colnames = [cv + '_' + str(modality) for cv in colnames for modality in le_dict[cv].classes_]
    # for cv in colnames:
    #     for modality in le_dict[cv].classes_:
    #         dummy_colnames.append(cv + '_' + modality)

    return X, dummy_colnames, enc
to_dummify = ["Pclass"]
X_dummy, dummy_columns, dummy_encoder = dummify_features(train[to_dummify])
X_dummy = pd.DataFrame(X_dummy.toarray(), columns=dummy_columns)

In [ ]:
- Faire la meme chose avec un modele ensembliste